In [8]:
import pandas as pd
import numpy as np
import os
from pytradersan import constants
from pytradersan.helpers import standardize_transactions
from pytradersan.portfolio import Portfolio
from pathlib import Path
import glob
from collections import defaultdict
import requests
import json

from pytradersan.secrets.schwab import SCHWAB_BEARER_TOKEN

In [6]:
os.getcwd()

'/home/sandeep/projects/pytradersan/scripts/notebooks'

In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
headers = {
    'accept': 'application/json', 
    'Authorization': SCHWAB_BEARER_TOKEN,
    }

url = 'https://api.schwabapi.com/trader/v1/accounts/accountNumbers'

r = requests.get(url, headers=headers)

In [50]:
encrypted_account = r.json()[0]['hashValue']

In [66]:
encrypted_account

'68F76B8241AAEEAD55C840BAFAFFC16F340DC3FE3919886A9E20EC8E52480041'

In [ ]:
headers = {
    'accept': 'application/json', 
    'Authorization': SCHWAB_BEARER_TOKEN,
    }

params = {
    'startDate': '2024-01-01T00:00:00.000Z',
    'endDate': '2024-12-31T00:00:00.000Z',
    'types': 'TRADE',
}

url = f'https://api.schwabapi.com/trader/v1/accounts/{encrypted_account}/transactions'
r2 = requests.get(url, headers=headers, params=params)

In [108]:
# raw_transactions = r.json()
# r.url
r2

<Response [401]>

In [77]:
len(raw_transactions)

86

In [79]:
raw_transactions[-1].keys()

dict_keys(['activityId', 'time', 'accountNumber', 'type', 'status', 'subAccount', 'tradeDate', 'positionId', 'orderId', 'netAmount', 'transferItems'])

In [106]:
txns = pd.read_json(json.dumps(raw_transactions))

/tmp/ipykernel_503451/1293870510.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  txns = pd.read_json(json.dumps(raw_transactions))


In [96]:
txns.sort_values(by='netAmount', ascending=False)

,activityId,time,accountNumber,type,status,subAccount,tradeDate,positionId,orderId,netAmount,transferItems
39,86068541987,2024-09-18T14:37:57+0000,75365576,TRADE,VALID,MARGIN,2024-09-18T14:37:57+0000,2474152097,1001625616762,15490.55,"[{'instrument': {'assetType': 'CURRENCY', 'sta..."
22,88899812804,2024-11-21T15:09:59+0000,75365576,TRADE,VALID,MARGIN,2024-11-21T15:09:59+0000,2474152101,1002281209763,15386.35,"[{'instrument': {'assetType': 'CURRENCY', 'sta..."
21,88906958522,2024-11-21T15:45:45+0000,75365576,TRADE,VALID,MARGIN,2024-11-21T15:45:45+0000,2474152102,1002283079911,14316.48,"[{'instrument': {'assetType': 'CURRENCY', 'sta..."
37,86126686975,2024-09-19T13:55:50+0000,75365576,TRADE,VALID,MARGIN,2024-09-19T13:55:50+0000,2474152097,1001638683085,11199.68,"[{'instrument': {'assetType': 'CURRENCY', 'sta..."
38,86126686970,2024-09-19T13:55:50+0000,75365576,TRADE,VALID,MARGIN,2024-09-19T13:55:50+0000,2474152097,1001638683085,11199.67,"[{'instrument': {'assetType': 'CURRENCY', 'sta..."
...,...,...,...,...,...,...,...,...,...,...,...
16,89794955261,2024-12-13T13:59:23+0000,75365576,TRADE,VALID,MARGIN,2024-12-13T13:59:23+0000,2679920816,1002434731133,-4750.00,"[{'instrument': {'assetType': 'CURRENCY', 'sta..."
4,90100119574,2024-12-19T20:48:28+0000,75365576,TRADE,VALID,MARGIN,2024-12-19T20:48:28+0000,2508357044,1002476744756,-4864.50,"[{'instrument': {'assetType': 'CURRENCY', 'sta..."
40,85267050592,2024-08-28T16:30:01+0000,75365576,TRADE,VALID,MARGIN,2024-08-28T16:30:01+0000,2556420671,1001443459379,-5562.50,"[{'instrument': {'assetType': 'CURRENCY', 'sta..."
7,89940850231,2024-12-17T15:47:36+0000,75365576,TRADE,VALID,MARGIN,2024-12-17T15:47:36+0000,2684754316,1002455486019,-6000.00,"[{'instrument': {'assetType': 'CURRENCY', 'sta..."


In [104]:
txns.loc[0, 'transferItems'][4]

{'instrument': {'assetType': 'EQUITY',
  'status': 'ACTIVE',
  'symbol': 'AMD',
  'instrumentId': 1137441,
  'closingPrice': 103.22,
  'type': 'COMMON_STOCK'},
 'amount': 20.0,
 'cost': -2499.79,
 'price': 124.9895,
 'positionEffect': 'OPENING'}

In [36]:
origin = Path('../..')

schwab_files = glob.glob(str(origin / 'data' / 'schwab*.csv'))
print(schwab_files)

# Load raw Schwab transactions
schwab_transactions = defaultdict(dict)
for schwab_file in schwab_files:
    name = schwab_file.split('/')[-1].split('.csv')[0]
    print(f'Loading {schwab_file} with key {name}')
    schwab_transactions[name] = pd.read_csv(schwab_file)


['../../data/schwab576_20250328.csv']
Loading ../../data/schwab576_20250328.csv with key schwab576_20250328


In [ ]:
for 